## Sistema de recomendación con Google_maps API

En este proyecto vamos a generar un sistema de recomendación del tipo *content base* que basado en el rating que da un usuario a ciertos lugares que visitó previamente, sugiere otras lugares a visitar. Para el caso vamos a buscarnos donde tomar un buen cafe en una ciudad que me encanta (Manizales).Para ello usaremos la API de googlemaps a traves de la cual accederemos a la poderosa base de datos que posee Googlemaps, sin embargo, se debe tener en cuenta que para acceder a información como *rating* de otros usuarios, *horario de atención* o las imagenes del establecimiento, se debe acceder de una manera especifica que genera costos al usuario. Por lo tanto solo usamos la información a la que se puede accesar a traves de la versión gratuita de googleplatform 

In [1]:
# instalamos la bibliteca disponible en python para google maps
pip install googlemaps

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
# instalamos un paquete que nos permite visualizar un poco mejor los archivos JSON que recibimos de la API
pip install prettyprint

Note: you may need to restart the kernel to use updated packages.


In [27]:
# cargamos los paquetes que vamos a utilizar
import googlemaps
import pprint
import time # lo necesitamos principalmente para crear un delay que nos permite extraer una mayor cantidad de datos desde la api 
import pandas as pd

In [28]:
#Define our CLient
gmaps= googlemaps.Client(key = my_key )

In [54]:
##### definir busqueda
place = gmaps.places_nearby(location = '5.0560188,-75.4859003', radius =  600, type= 'cafe')
#places_res = gmaps.places_nearby(page_token = place['next_page_token'])

In [40]:
# veamos un poco la estructura de los datos 
pprint.pprint(place)


{'html_attributions': [],
 'next_page_token': 'CqQCGgEAAHSZFxE-PRXd1L3xxnSzy_Q4n5hlsH1JdNjsYUvJoUFNRgdVXyTuIEChW0CCkSm17xKaNPkdfE7OQ6kWKlX-KbTbo_4xokg5nk49stWuchd2MU6h5UttyH19REVTahSCTzvjuR2Rlla31gokRafyv5mQD_gh6RRMLjRQYJVMQm2XYjhP8-DF4IIlvCb5QkqvTSWzH2P6xufFv_RURF1ZUPGdYbJVBIUaqeB5vz1Y3lQ0VdQoVHXmyRXjwFF4cDLzYgQcITi1KIfs57vLGdXVuho9HdgfdJHwcFpG6XuLl6djDWkZ7H4l2uwV7y-k7orZosJZWLFGQ2362_Z3DbucIWmG1XNGy6rYYM79bZbIxTUbJuCaVP2v3KCeDzUPJ5bYBRIQj6qA5qKWu1nFIxBiXTHFsxoUlx-tqtZmISZOEobu9k-0r_7K-EE',
 'results': [{'geometry': {'location': {'lat': 5.056276899999999,
                                        'lng': -75.4865975},
                           'viewport': {'northeast': {'lat': 5.057644580291502,
                                                      'lng': -75.4853068697085},
                                        'southwest': {'lat': 5.054946619708497,
                                                      'lng': -75.4880048302915}}},
              'icon': 'https://maps.gstatic.com/mapf

In [41]:
# vamos a tomar el next token para obtener mas resultados
resultados2 =gmaps.places_nearby(page_token =place['next_page_token'])
#pprint.pprint(resultados2['next_page_token'])
time.sleep(5)
resultados3 = gmaps.places_nearby(page_token = resultados2['next_page_token'])





In [42]:
# hacemos una lista con los 60 resultados que obtuvimos 
resultados_busqueda = [place, resultados2, resultados3]

## Algunas Funciones a Usar

Veamos algunas de las funciones que nos permitirn preprocesar la informacion que viene un poco desordenada desdnos permite exe la api.
1. La función "**resultados_lugares**" nos va extraer de los datos iniciales que nos arroja la api solo el nombre y las caracteristicas del establecimiento.
2. La función "**servicios_local**" nos va a permitir codificar los servicios que ofrece cada tienda. De modo que si tiene el servicio se escribe 1 o sino 0.
3. La función "**Obtener_valores**" nos va a permitir crear un diccionario para cada establecimiento con la informacion codificada, teniendo como campos los servicios que ofrece, estos son:
café, food, restaurante, bakey, store. 

In [43]:
def resultados_lugares(lugares):
    ''' definimos una función que para cada establecimiento encuentra el nombre y tipo de negocio'''
    total_sitios = []
    for lugar in lugares['results']:
        # definamos la info que queremos de cada lugar
        nombre = lugar['name']
        tipo_negocio = lugar['types']
        # hacemos ahora una consulta para estos datos
        # diccionario con los datos por lugar 
        dict_lugar = {}
        dict_lugar['nombre']=nombre
        dict_lugar['tipo_negocio'] = tipo_negocio
        total_sitios.append(dict_lugar)
    return total_sitios
        

In [44]:
# veamos como ejemplo como la función resultados_lugares nos permite ver el nomnre y tipo de servicios que ofrece cada establecimiento 
# que se obtuvo en el primer conjunto de resultados que nos arroja la api
resultado = resultados_lugares(place)
for i in resultado:
    print(i)

{'nombre': 'Juan Valdez Café', 'tipo_negocio': ['cafe', 'restaurant', 'food', 'point_of_interest', 'establishment']}
{'nombre': 'Pan Extra', 'tipo_negocio': ['bakery', 'cafe', 'food', 'point_of_interest', 'store', 'establishment']}
{'nombre': 'Kaffe Florida', 'tipo_negocio': ['restaurant', 'florist', 'cafe', 'food', 'point_of_interest', 'store', 'establishment']}
{'nombre': 'Samay Coffee Orgánico', 'tipo_negocio': ['cafe', 'food', 'point_of_interest', 'establishment']}
{'nombre': 'Churrisimo.CO - MANIZALES', 'tipo_negocio': ['bakery', 'cafe', 'restaurant', 'food', 'point_of_interest', 'store', 'establishment']}
{'nombre': "Dunkin' Donuts", 'tipo_negocio': ['cafe', 'food', 'point_of_interest', 'establishment']}
{'nombre': 'La iguana', 'tipo_negocio': ['cafe', 'food', 'point_of_interest', 'establishment']}
{'nombre': 'Plaza Café', 'tipo_negocio': ['cafe', 'food', 'point_of_interest', 'establishment']}
{'nombre': 'Café Iguana', 'tipo_negocio': ['cafe', 'food', 'point_of_interest', 'establ

In [45]:
def servicios_local(local):
    servicios = ['cafe', 'food', 'restaurant','bakery', 'store']
    serv = local['tipo_negocio']
    for i in servicios:
        if i in serv:
            pos = servicios.index(i)
            serv.remove(i)
            serv.insert(pos, 1)
        else:
            pos = servicios.index(i)
            serv.insert(pos, 0)
    lista_clean = serv
    lista_clean = [a for a in lista_clean if isinstance(a, int)]
    local['tipo_negocio'] = lista_clean
    return local


In [51]:
''' a modo de ejemplo veamos como quedan codificados los datos para los establecimientos de la primera lista de resultados que nos arroja la api'''
ejemplo = resultados_lugares(place)
for item in ejemplo:
    servicios_local(item)
    print(item)

{'nombre': 'Juan Valdez Café', 'tipo_negocio': [1, 1, 1, 0, 0]}
{'nombre': 'Pan Extra', 'tipo_negocio': [1, 1, 0, 1, 1]}
{'nombre': 'Kaffe Florida', 'tipo_negocio': [1, 1, 1, 0, 1]}
{'nombre': 'Samay Coffee Orgánico', 'tipo_negocio': [1, 1, 0, 0, 0]}
{'nombre': 'Churrisimo.CO - MANIZALES', 'tipo_negocio': [1, 1, 1, 1, 1]}
{'nombre': "Dunkin' Donuts", 'tipo_negocio': [1, 1, 0, 0, 0]}
{'nombre': 'La iguana', 'tipo_negocio': [1, 1, 0, 0, 0]}
{'nombre': 'Plaza Café', 'tipo_negocio': [1, 1, 0, 0, 0]}
{'nombre': 'Café Iguana', 'tipo_negocio': [1, 1, 0, 0, 0]}
{'nombre': 'Juan Valdez Café', 'tipo_negocio': [1, 1, 0, 0, 1]}
{'nombre': 'Cereza Café', 'tipo_negocio': [1, 1, 0, 0, 0]}
{'nombre': 'Cafe Cava Santa Manizales', 'tipo_negocio': [1, 1, 0, 0, 0]}
{'nombre': 'Criossant Casa Lúker', 'tipo_negocio': [1, 1, 0, 0, 0]}
{'nombre': 'BEBIDAS MICHELIN', 'tipo_negocio': [1, 1, 0, 0, 0]}
{'nombre': 'Juan Valdez Café', 'tipo_negocio': [1, 1, 0, 0, 1]}
{'nombre': 'Mecato y Sazon', 'tipo_negocio': [1,

In [47]:
''' ahora cconstruimos una funcion que nos permite obtener un diccionario con los datos codificados y los campos cafe, food, restaurante, bakery, store'''
def obtener_valores(elemento): 
    dic_valores ={}
    try:
        dic_valores['nombre'] = elemento['nombre']
    except:
        dic_valores['nombre'] = None
    try:
        dic_valores['cafe'] = elemento['tipo_negocio'][0]
    except:
        dic_valores['cafe'] = 0
    try:
        dic_valores['food']= elemento['tipo_negocio'][1]
    except:
        dic_valores['food'] = 0
    try:
        dic_valores['restaurante'] = elemento['tipo_negocio'][2]
    except:
        dic_valores['restaurante'] = 0
    try:
        dic_valores['bakery'] = elemento['tipo_negocio'][3]
    except:
        dic_valores['bakery'] = 0
    try:
        dic_valores['store'] = elemento['tipo_negocio'][4]
    except:
        dic_valores['stores'] = 0
    
    return dic_valores
        


In [55]:
# a modo de ejemplo veamos el dataframe que obtenemos para el primer listado de resultados
ejemplo = resultados_lugares(place)
dict_total = []
for elemento in ejemplo:
    elemento_codi = servicios_local(elemento)
    dict_valores = obtener_valores(elemento_codi)
    dict_total.append(dict_valores)
    df = pd.DataFrame(dict_total)


In [58]:
print(df.head(8))

                      nombre  cafe  food  restaurante  bakery  store
0           Juan Valdez Café     1     1            1       0      0
1                  Pan Extra     1     1            0       1      1
2              Kaffe Florida     1     1            1       0      1
3      Samay Coffee Orgánico     1     1            0       0      0
4  Churrisimo.CO - MANIZALES     1     1            1       1      1
5             Dunkin' Donuts     1     1            0       0      0
6                  La iguana     1     1            0       0      0
7                 Plaza Café     1     1            0       0      0


In [ ]:
# ahora implementamos todo el proceso para el total de los resultados a los que nos permite acceder la api.
# NOTA : recordemos que la api solo nos permite acceder a 60 resultados. estos para la version gratuita qued es la usada en el proyecto.
diccionarios =[]
for item in resultados_busqueda:
    resultado = resultados_lugares(item)
    dict_total = []
    for elemento in resultado:
        elemento_codi = servicios_local(elemento)
        dict_valores = obtener_valores(elemento_codi)
        dict_total.append(dict_valores)
    df = pd.DataFrame(dict_total)
    diccionarios.append(df)
res = pd.concat(diccionarios)

In [118]:
res = res.reset_index(drop = True)

In [145]:
print(res.head(8))

                      nombre  cafe  food  restaurante  bakery  store
0           Juan Valdez Café     1     1            1       0      0
1                  Pan Extra     1     1            0       1      1
2              Kaffe Florida     1     1            1       0      1
3      Samay Coffee Orgánico     1     1            0       0      0
4  Churrisimo.CO - MANIZALES     1     1            1       1      1
5             Dunkin' Donuts     1     1            0       0      0
6                  La iguana     1     1            0       0      0
7                 Plaza Café     1     1            0       0      0


In [115]:
print(res.tail())

                                   nombre  cafe  food  restaurante  bakery  \
17                      Origen Coffe Club     1     1            0       0   
18                       Villa Clara Cafe     1     1            0       0   
19  Buñuelos y Más (Multicentro Estrella)     1     1            0       0   
0                             Goyi's Cafe     1     1            0       0   
1                          Jengibre Salón     1     1            1       1   

    store  
17      0  
18      1  
19      0  
0       0  
1       1  


In [126]:
print(res['nombre'])

0                                      Juan Valdez Café
1                                             Pan Extra
2                                         Kaffe Florida
3                                 Samay Coffee Orgánico
4                             Churrisimo.CO - MANIZALES
5                                        Dunkin' Donuts
6                                             La iguana
7                                            Plaza Café
8                                           Café Iguana
9                                      Juan Valdez Café
10                                          Cereza Café
11                            Cafe Cava Santa Manizales
12                                 Criossant Casa Lúker
13                                     BEBIDAS MICHELIN
14                                     Juan Valdez Café
15                                       Mecato y Sazon
16                                     Café Terraza Bar
17                                           Cre

**SISTEMA DE RECOMENDACION BASADO EN CONTENIDOS**

Ya que tenemos un dataframe con los datos de los establecimientos y sus servicios, pasamos a ahora a construir un sistema de recomendación de tipo *base content*

Para ello seguiremos los siguientes pasos:

**INPUT USER**
input con ls datos del usuario donde indica los sitios a los que ha asistido y la putuación que les ha asignado a dichos sitios

In [244]:
Sitios_input =[ {'nombre':'Jengibre Salón', 'rating':4.8 },
               {'nombre':'Kaffe Florida', 'rating':4.9 },
               {'nombre':'Pan Extra', 'rating':5 },
               {'nombre': 'Plaza Café','rating':1.5}] 
sitios_input = pd.DataFrame(Sitios_input)
print(sitios_input)

           nombre  rating
0  Jengibre Salón     4.8
1   Kaffe Florida     4.9
2       Pan Extra     5.0
3      Plaza Café     1.5


In [253]:
# vamos ahora a filtrar nuestro dataset de todos los lugares para obtener los indices de los indices de los sitios que rankeamos
# y lo agregamos al input anterior
sitios =pd.DataFrame(res[res['nombre'].isin(sitios_input['nombre'].tolist())])
sitios_input = sitios_input.set_index('nombre')
sitios_input = sitios_input.reindex(index =sitios['nombre'])
sitios_input = sitios_input.reset_index()
rating = sitios_input['rating']
print(rating)



0    5.0
1    4.9
2    1.5
3    4.8
Name: rating, dtype: float64


In [268]:

ind = sitios.index
sitios['ind'] = ind
sitios['rating'] = list(rating)
sitios.reset_index(drop = True)
print(sitios)
#sitios_input= pd.DataFrame(Sitios_input, index = [i for i in indices])
#print(sitios_input)
#sitios_input['id'] = indices
# suprimiendo las columnas cafe,...,store
#sitios_input = sitios_input.drop('cafe', 1).drop('food',1).drop('restaurante',1).drop('bakery',1).drop('store',1)
 




           nombre  cafe  food  restaurante  bakery  store  ind  rating
0       Pan Extra     1     1            0       1      1    0     5.0
1   Kaffe Florida     1     1            1       0      1    1     4.9
2      Plaza Café     1     1            0       0      0    2     1.5
3  Jengibre Salón     1     1            1       1      1    3     4.8


In [270]:
#matriz_locales = sitios.iloc[:, 1:6]
matriz_locales = sitios.drop('nombre', 1).drop('rating',1).drop('ind', 1)
print(matriz_locales)

   cafe  food  restaurante  bakery  store
0     1     1            0       1      1
1     1     1            1       0      1
2     1     1            0       0      0
3     1     1            1       1      1


In [275]:
perfil_usuario = matriz_locales.transpose().dot(rating) 
print(perfil_usuario)

cafe           16.2
food           16.2
restaurante     9.7
bakery          9.8
store          14.7
dtype: float64


In [284]:
# tomemos de los resultados obtenidos al scrapear la api de google, los valores codificados para los servicios
Tabla_locales = res.drop('nombre', 1)

# Hagamos una lista con los sitios sugeridos
Tabla_recomendados = ((Tabla_locales*perfil_usuario).sum(axis = 1 ) )/ (perfil_usuario.sum())

In [286]:
Tabla_recomendados = Tabla_recomendados.sort_values(ascending = False) 
print(Tabla_recomendados)

41    1.000000
4     1.000000
1     0.854354
23    0.852853
2     0.852853
17    0.852853
20    0.707207
33    0.707207
31    0.707207
38    0.707207
16    0.707207
15    0.707207
14    0.707207
9     0.707207
0     0.632132
28    0.486486
13    0.486486
39    0.486486
3     0.486486
37    0.486486
5     0.486486
34    0.486486
7     0.486486
8     0.486486
27    0.486486
26    0.486486
25    0.486486
24    0.486486
22    0.486486
21    0.486486
40    0.486486
19    0.486486
18    0.486486
32    0.486486
29    0.486486
35    0.486486
30    0.486486
12    0.486486
11    0.486486
10    0.486486
36    0.486486
6     0.486486
dtype: float64


In [296]:
# Finamente los sitios recomendados son:
index = Tabla_recomendados.index
sitios_recomendados = res.iloc[index,:][0:9]

In [297]:
print(sitios_recomendados)

                                  nombre  cafe  food  restaurante  bakery  \
41                        Jengibre Salón     1     1            1       1   
4              Churrisimo.CO - MANIZALES     1     1            1       1   
1                              Pan Extra     1     1            0       1   
23             KÁVA Beer & Coffee Lounge     1     1            1       0   
2                          Kaffe Florida     1     1            1       0   
17                            Crepe Cone     1     1            1       0   
20  Caferatto Café especial en Manizales     1     1            0       0   
33     La Malteaderia - Malteadas y Café     1     1            0       0   
31                      Azrael Café Gato     1     1            0       0   

    store  
41      1  
4       1  
1       1  
23      1  
2       1  
17      1  
20      1  
33      1  
31      1  
